# Experiment: _Node weights_

In [ ]:
from notebook_prelude import *

In [ ]:
EXPERIMENT_NAME = 'experiment_node_weights'
df = results_helper.get_experiments_by_names([EXPERIMENT_NAME], fetch_predictions=True)
df_ = results_helper.get_results(filter_out_non_complete_datasets=False, fetch_predictions=True, include_filter='__graph__')
df = df.append(df_).reset_index()
for x in ['graph__fast_wl__node_weight_iteration_weight_function', 'graph__fast_wl__node_weight_iteration_weight_function']:
    df[x + '_'] = df[x].apply(lambda x: x.__name__ if callable(x) else '-')

In [ ]:
df.groupby(['dataset', 'graph__fast_wl__node_weight_function']).mean_test_f1_macro.max().to_frame()

In [ ]:
df.groupby(['dataset', 'normalizer']).mean_test_f1_macro.max().to_frame().unstack()

In [ ]:
attr = 'mean_test_f1_macro'
attr = 'prediction_score_f1_macro'
df_ = df.groupby(['dataset', 'graph__fast_wl__node_weight_function'])[attr].max().to_frame().unstack()
df_['difference_non_vs_weight'] = (df_[attr, 'none'] - df_[attr, 'nxgraph_degrees_metric'])
#df_['mean_test_f1_macro', 'none']
#df_.columns = df_.columns.droplevel()
#df_.columns = ['plain', 'degrees_node_weights', 'difference']

#df__ = df_[['plain', 'degrees_node_weights']]
#print(df__.to_latex(float_format="%.3f"))
df_

In [ ]:
pprint(df.columns.values.tolist())

## Other graph datasets

_Mutag_, _Enzymes_, etc.

In [ ]:
df = results_helper.get_experiments_by_names(['experiment_graph_extra_plain', 'experiment_graph_extra_node_weights'], fetch_predictions=True)
df['graph__fast_wl__node_weight_function'] = df.graph__fast_wl__node_weight_function.apply(lambda x: x.__name__ if callable(x) else x)

In [ ]:
#attr = 'prediction_score_f1_macro'
attr = 'mean_test_f1_macro'
attr = 'prediction_score_f1_macro'
#attr = 'prediction_score_accuracy'
best = df.groupby(['dataset', 'graph__fast_wl__node_weight_function'])[attr].max().to_frame().unstack()
best.columns = best.columns.droplevel()
best.rename(columns={'-': 'plain'}, inplace=True)

best = best[(best.plain != '-') & (best.adj_degrees_metric_max != '-')]
best = best[best.adj_degrees_metric_max != None]
best['difference'] = best.adj_degrees_metric_max - best.plain
#print(best.to_latex())
best

In [ ]:
df_ = df[df[attr] != '-']
df_[attr] = df_[attr].astype(np.float64)
best_ = df_.loc[df_.groupby(['dataset', 'graph__fast_wl__node_weight_function'])['mean_test_f1_macro'].idxmax()]

prediction_file = best_[best_.dataset == 'MSRC_9'].iloc[2].prediction_file
rs = get_pickle(prediction_file)
rs = rs['results']
X_test = rs['X_test']
np.array(rs['Y_real']) == rs['Y_pred']
#best = df_.groupby(['dataset', 'graph__fast_wl__node_weight_function'])[attr].idxmax()
#df_
#best[best.dataset == 'MSRC_9']

In [ ]:
DATASET = 'MSRC_9'
tasks = [x for x in experiments.get_filtered_tasks(task_type='graph_extra', dataset=DATASET) if DATASET in x.name]
assert len(tasks) == 1
task = tasks[0]

In [ ]:
X, Y, estimator, params = task.fn()
experiment_config = experiment_helper.get_experiment_config('configs/experiments/graph_extra/experiment_graph_extra_node_weights.yaml')
param_grid = experiment_helper.prepare_param_grid(task, params, experiment_config)
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, stratify=Y, test_size=0.2, random_state=40)
cv = sklearn.model_selection.StratifiedKFold(n_splits=4)
gscv = sklearn.model_selection.GridSearchCV(estimator, param_grid, scoring='f1_macro', refit=True, cv=cv, verbose=1)
score = gscv.fit(X_train, Y_train)
print(score)

In [ ]:
Y_test_ = score.best_estimator_.predict(X_test)
sklearn.metrics.f1_score(Y_test, Y_test_, average='macro')

In [ ]:

np.where(gscv.predict(X_test) == rs['Y_real']), sklearn.metrics.f1_score(rs['Y_real'], gscv.predict(X_test), average='macro')

In [ ]:
score.best_score_

In [ ]:

from utils import significance_test_utils
attr = 'graph__fast_wl__node_weight_function'
confidences = collections.defaultdict(list)
for dataset, df_ in df[df.prediction_score_f1_macro !='-'].groupby('dataset'):
    assert attr in df_.columns
    print(dataset)
    df_['prediction_score_f1_macro'] = df_.prediction_score_f1_macro.astype(np.float64)
    best_ = df_.loc[df_.groupby(attr).prediction_score_f1_macro.idxmax()]
    best_ = best_[best_.graph__fast_wl__node_weight_function != 'adj_degrees_metric']
    #display(df_[attr].unique())
    if len(best_) != 2:
        print('\tWrong number of models: {}, expected=2. Skipping'.format(len(best_)))
        continue
    prediction_filenames = [best_.loc[best_[attr] == name].iloc[0].prediction_file for name in ['-', 'adj_degrees_metric_max']]
    diffs, score_a, score_b, global_difference, confidence = results_helper.calculate_significance(prediction_filenames[0], prediction_filenames[1])
    
    for k, v in [('Score A', score_a), ('Score B', score_b), ('Difference', global_difference), ('Confidence', confidence)]:
        print('\t{:20} {:9.4f}'.format(k, v))
    print()
    confidences['dataset'].append(dataset)
    confidences['confidence'].append(confidence)
    best.loc[dataset, 'confidence'] = confidence


In [ ]:
best

### Statistics about benchmark graphs

In [ ]:
data = collections.defaultdict(list)
for dataset in log_progress_nb(graph_helper.get_all_graph_benchmark_dataset_names()):
    X, Y = graph_helper.get_mutag_enzyme_graphs(dataset)
    assert len(X) == len(Y)
    nodes = list(chain.from_iterable([labels for _, labels in X]))
    unique_nodes = set(nodes)
    num_edges = sum([adj.nonzero()[0].shape[0] for adj, labels in X])
    data['dataset'].append(dataset)
    data['num_nodes'].append(len(nodes))
    data['num_unique_nodes'].append(len(unique_nodes))
    data['num_edges'].append(num_edges)
    data['num_graphs'].append(len(X))
    data['num_classes'].append(len(set(Y)))

In [ ]:
df_graph_statistics = pd.DataFrame(data)
df_graph_statistics[df_graph_statistics.num_graphs > 400]